In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)
from spyne.application import Application
from spyne.decorator import srpc
from spyne.service import ServiceBase
from spyne.model.primitive import Integer
from spyne.model.primitive import Unicode
from spyne.model.complex import Iterable
from spyne.protocol.http import HttpRpc
from spyne.protocol.json import JsonDocument
from spyne.server.wsgi import WsgiApplication
import hashlib
import redis

class DiminishedTriadService(ServiceBase):

    def __init__(self):
        print "I am being instantiated !!!!"
        
    @srpc(Unicode, _returns=Unicode)
    def get(key):
        backs = getNextNBackends(key)
        #r = redis.StrictRedis(host='localhost', port=30001, db=0)
        print "lenght of backs : ", len(backs)
        print "backs : ", backs
        print hash_to_back_map
        master_add = backs[0].split(":")[0]
        master_port = backs[0].split(":")[1]
        r = redis.StrictRedis(host=master_add, port=master_port, db=0)   
        return r.get(key)
        #return "GET"
    
    @srpc(Unicode,Unicode, _returns=Unicode)
    def set(key,value):
        #r = redis.StrictRedis(host='localhost', port=30001, db=0)
        backs = getNextNBackends(key)
        print "lenght of backs : ", len(backs)
        print "backs : ", backs
        print hash_to_back_map
        master_add = backs[0].split(":")[0]
        master_port = backs[0].split(":")[1]
        r = redis.StrictRedis(host=master_add, port=master_port, db=0)     
        return r.set(key, value)
        
        #return "SET"

############################### End of Class DiminishedTriadService ########################################

NUMBACKENDS = 100 # this determines what we are modding by
NUMBACKUPS = 1

def getRingPos(key):
    return int(hashlib.sha1(key).hexdigest(), 16) % NUMBACKENDS

hash_to_back_map = {
                            getRingPos("127.0.0.1:30001"):"127.0.0.1:30001",
                            getRingPos("127.0.0.1:30002"):"127.0.0.1:30002",
                            getRingPos("127.0.0.1:30003"):"127.0.0.1:30003"
                    }

def getNextNBackends(key):
    keys = hash_to_back_map.keys()
    print "keys : ", keys
    keys.append(getRingPos(key))
    keys = sorted(keys)
    index = keys.index(getRingPos(key))
    out = []
    for i in range(1,NUMBACKUPS+2):
        if keys[(index + i)%len(keys)] in hash_to_back_map: #handle duplicate keys TODO
            out.append(hash_to_back_map[keys[(index + i)%len(keys)]])
    print "getNextNBackends for key : ",key," returning : {",out,"}"
    return out

if __name__ == '__main__':
    from wsgiref.simple_server import make_server
    #logging.basicConfig(level=logging.DEBUG)
    #logging.getLogger('spyne.protocol.xml').setLevel(logging.DEBUG)
    application = Application([DiminishedTriadService],
        tns='DiminishedTriadService',
        in_protocol=HttpRpc(validator='soft'),
        out_protocol=JsonDocument()
    )
    wsgi_app = WsgiApplication(application)
    server = make_server('127.0.0.1', 8000, wsgi_app)
    server.serve_forever()
    
    
    